In [1]:
import numpy as np
from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.linalg import spsolve
from numba import jit, prange
import time

# ========================
# 参数配置
# ========================
nx, ny = 50, 100          # 单元数量
width, height = 50.0, 100.0  # 模型尺寸 (mm)
E = 210e3                 # 弹性模量 (MPa)
nu = 0.3                  # 泊松比
thickness = 1.0           # 厚度 (mm)
applied_stress = -6.0     # 施加应力 (MPa)

# 裂纹区域定义 (示例)
crack_area = {
    "x_range": (20.0, 30.0),
    "y_range": (40.0, 60.0),
    "E_ratio": 1e-6       # 弹性模量衰减系数
}

# ========================
# 网格生成
# ========================
@jit(nopython=True, cache=True)
def generate_mesh(nx, ny, width, height):
    num_nodes_x = nx + 1
    num_nodes_y = ny + 1
    nodes = np.zeros((num_nodes_x * num_nodes_y, 2))
    dx = width / nx
    dy = height / ny
    
    for j in prange(num_nodes_y):
        for i in prange(num_nodes_x):
            idx = j * num_nodes_x + i
            nodes[idx, 0] = i * dx
            nodes[idx, 1] = j * dy
    
    elements = np.zeros((nx * ny, 4), dtype=np.int64)
    for j in prange(ny):
        for i in prange(nx):
            elem_id = j * nx + i
            n0 = j * (nx + 1) + i
            elements[elem_id] = [n0, n0+1, n0+1 + (nx+1), n0 + (nx+1)]
    
    return nodes, elements

# ========================
# 材料属性分配
# ========================
@jit(nopython=True, parallel=True)
def assign_material_properties(nodes, elements, crack):
    E_values = np.full(len(elements), E)
    for elem_id in prange(len(elements)):
        node_ids = elements[elem_id]
        x_center = np.mean(nodes[node_ids, 0])
        y_center = np.mean(nodes[node_ids, 1])
        
        if (crack["x_range"][0] <= x_center <= crack["x_range"][1] and
            crack["y_range"][0] <= y_center <= crack["y_range"][1]):
            E_values[elem_id] = E * crack["E_ratio"]
    return E_values

# ========================
# 刚度矩阵组装 (Numba加速)
# ========================
@jit(nopython=True, parallel=True, fastmath=True)
def assemble_stiffness(nodes, elements, E_values, dof):
    K_data = np.zeros((len(elements)*64, 3))  # 预分配内存 (每个单元最多贡献64项)
    gp = np.array([[-0.57735, -0.57735], [0.57735, -0.57735],
                   [0.57735, 0.57735], [-0.57735, 0.57735]])
    weights = np.ones(4)
    count = 0
    
    for elem_id in prange(len(elements)):
        elem = elements[elem_id]
        node_coords = nodes[elem]
        E_e = E_values[elem_id]
        D = E_e/(1-nu**2) * np.array([[1, nu, 0], [nu, 1, 0], [0, 0, (1-nu)/2]])
        
        Ke = np.zeros((8, 8))
        for gp_id in range(4):
            xi, eta = gp[gp_id]
            dN = 0.25 * np.array([
                [eta-1, 1-eta, 1+eta, -eta-1],
                [xi-1, -xi-1, xi+1, 1-xi]
            ])
            J = dN @ node_coords
            detJ = np.linalg.det(J)
            invJ = np.linalg.inv(J)
            B = np.zeros((3, 8))
            
            for i in range(4):
                dx = invJ @ dN[:, i]
                B[0, 2*i] = dx[0]
                B[1, 2*i+1] = dx[1]
                B[2, 2*i] = dx[1]
                B[2, 2*i+1] = dx[0]
            
            Ke += (B.T @ D @ B) * detJ * weights[gp_id] * thickness
        
        # 记录非零元素
        for i in range(4):
            for j in range(4):
                for di in range(2):
                    for dj in range(2):
                        row = 2*elem[i] + di
                        col = 2*elem[j] + dj
                        K_data[count] = (row, col, Ke[2*i+di, 2*j+dj])
                        count += 1
                        
    return K_data[:count]

# ========================
# 应力计算
# ========================
@jit(nopython=True, parallel=True)
def calculate_stress(U, nodes, elements, E_values):
    stress_matrix = np.zeros((ny, nx))
    gp = np.array([[0.0, 0.0]])  # 中心点积分
    
    for elem_id in prange(len(elements)):
        elem = elements[elem_id]
        u = np.concatenate([U[2*elem[i]:2*elem[i]+2] for i in range(4)])
        node_coords = nodes[elem]
        
        xi, eta = gp[0]
        dN = 0.25 * np.array([
            [eta-1, 1-eta, 1+eta, -eta-1],
            [xi-1, -xi-1, xi+1, 1-xi]
        ])
        J = dN @ node_coords
        invJ = np.linalg.inv(J)
        B = np.zeros((3, 8))
        
        for i in range(4):
            dx = invJ @ dN[:, i]
            B[0, 2*i] = dx[0]
            B[1, 2*i+1] = dx[1]
            B[2, 2*i] = dx[1]
            B[2, 2*i+1] = dx[0]
        
        strain = B @ u
        E_e = E_values[elem_id]
        D = E_e/(1-nu**2) * np.array([[1, nu, 0], [nu, 1, 0], [0, 0, (1-nu)/2]])
        stress = D @ strain
        
        s1 = (stress[0]+stress[1])/2 + np.sqrt(((stress[0]-stress[1])/2)**2 + stress[2]**2)
        stress_matrix[elem_id//nx, elem_id%nx] = s1
    
    return stress_matrix

# ========================
# 主程序
# ========================
if __name__ == "__main__":
    # 生成网格
    t0 = time.time()
    nodes, elements = generate_mesh(nx, ny, width, height)
    print(f"Mesh generated: {time.time()-t0:.2f}s")

    # 分配材料属性
    E_values = assign_material_properties(nodes, elements, crack_area)
    print(f"Material assigned: {time.time()-t0:.2f}s")

    # 组装刚度矩阵
    dof = 2 * len(nodes)
    K_data = assemble_stiffness(nodes, elements, E_values, dof)
    
    # 构建稀疏矩阵
    rows = K_data[:, 0].astype(int)
    cols = K_data[:, 1].astype(int)
    data = K_data[:, 2]
    K = csr_matrix((data, (rows, cols)), shape=(dof, dof))
    print(f"Stiffness matrix assembled: {time.time()-t0:.2f}s")

    # 边界条件（固定底边）
    bottom_nodes = np.where(nodes[:, 1] == 0.0)[0]
    fixed_dofs = np.concatenate([2*bottom_nodes, 2*bottom_nodes+1])
    
    # 载荷施加（顶部均布力）
    F = np.zeros(dof)
    top_edges = elements[np.where(nodes[elements[:, 3], 1] == height)[0]]
    for elem in top_edges:
        n2, n3 = elem[2], elem[3]
        length = np.linalg.norm(nodes[n3] - nodes[n2])
        F[2*n3+1] += applied_stress * length * thickness / 2
        F[2*n2+1] += applied_stress * length * thickness / 2

    # 处理边界条件
    K = K.tolil()
    for fd in fixed_dofs:
        K[fd, :] = 0
        K[:, fd] = 0
        K[fd, fd] = 1.0
    F[fixed_dofs] = 0.0
    K = K.tocsr()

    # 求解位移
    print("Solving system...")
    U = spsolve(K, F)
    print(f"Solution completed: {time.time()-t0:.2f}s")

    # 计算应力
    stress_matrix = calculate_stress(U, nodes, elements, E_values)
    print(f"Stress calculated: {time.time()-t0:.2f}s")

    # 输出结果
    print("\nStress Matrix Sample (5x5):")
    print(stress_matrix[:5, :5])
    print(f"\nMax principal stress: {np.max(stress_matrix):.2f} MPa")

Mesh generated: 0.75s


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at C:\Users\32383\AppData\Local\Temp\ipykernel_51640\1199281167.py (55)[0m
[1m
File "C:\Users\32383\AppData\Local\Temp\ipykernel_51640\1199281167.py", line 55:[0m
[1mdef assign_material_properties(nodes, elements, crack):
[1m    E_values = np.full(len(elements), E)
[0m    [1m^[0m[0m 

This error may have been caused by the following argument(s):
- argument 2: [1mCannot determine Numba type of <class 'dict'>[0m


In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.linalg import spsolve
from numba import jit, prange, typed
from numba.core import types
import time
import matplotlib.pyplot as plt

# ========================
# 参数配置
# ========================
nx, ny = 50, 100  # 单元数量
width, height = 50.0, 100.0  # 模型尺寸 (mm)
E = 210e3  # 弹性模量 (MPa)
nu = 0.3  # 泊松比
thickness = 1.0  # 厚度 (mm)
applied_stress = -6.0  # 施加应力 (MPa)

# ========================
# 网格生成
# ========================
@jit(nopython=True, cache=True)
def generate_mesh(nx, ny, width, height):
    num_nodes_x = nx + 1
    num_nodes_y = ny + 1
    nodes = np.zeros((num_nodes_x * num_nodes_y, 2))
    dx = width / nx
    dy = height / ny

    for j in range(num_nodes_y):
        for i in range(num_nodes_x):
            idx = j * num_nodes_x + i
            nodes[idx, 0] = i * dx
            nodes[idx, 1] = j * dy

    elements = np.zeros((nx * ny, 4), dtype=np.int64)
    for j in range(ny):
        for i in range(nx):
            elem_id = j * nx + i
            n0 = j * (nx + 1) + i
            elements[elem_id] = [n0, n0 + 1, n0 + 1 + (nx + 1), n0 + (nx + 1)]

    return nodes, elements

# ========================
# 材料属性分配
# ========================
@jit(nopython=True, parallel=True)
def assign_material_properties(nodes, elements, crack_x_range, crack_y_range, crack_E_ratio):
    E_values = np.full(len(elements), E)
    for elem_id in prange(len(elements)):
        node_ids = elements[elem_id]
        x_center = np.mean(nodes[node_ids, 0])
        y_center = np.mean(nodes[node_ids, 1])

        if (crack_x_range[0] <= x_center <= crack_x_range[1] and
                crack_y_range[0] <= y_center <= crack_y_range[1]):
            E_values[elem_id] = E * crack_E_ratio
    return E_values

# ========================
# 刚度矩阵组装 (Numba加速)
# ========================
@jit(nopython=True, parallel=True, fastmath=True)
def assemble_stiffness(nodes, elements, E_values, num_dofs):
    max_entries = len(elements) * 64 * 1.5
    rows = np.zeros(int(max_entries), dtype=np.int64)
    cols = np.zeros(int(max_entries), dtype=np.int64)
    data = np.zeros(int(max_entries), dtype=np.float64)

    gauss_points = np.array([[-0.57735, -0.57735], [0.57735, -0.57735],
                             [0.57735, 0.57735], [-0.57735, 0.57735]])
    weights = np.ones(4)
    count = 0

    for elem_id in prange(len(elements)):
        elem = elements[elem_id]
        node_coords = nodes[elem].copy()
        E_e = E_values[elem_id]
        D = E_e / (1 - nu**2) * np.array([[1, nu, 0], [nu, 1, 0], [0, 0, (1 - nu) / 2]])

        Ke = np.zeros((8, 8))
        for gp_id in range(4):
            xi, eta = gauss_points[gp_id]
            dN = 0.25 * np.array([
                [eta - 1, 1 - eta, 1 + eta, -eta - 1],
                [xi - 1, -xi - 1, xi + 1, 1 - xi]
            ])
            J = dN @ node_coords

            detJ = np.linalg.det(J)
            if abs(detJ) < 1e-12:
                print("Warning: detJ is close to zero!")

            invJ = np.linalg.inv(J)
            B = np.zeros((3, 8))

            for i in range(4):
                dx = invJ @ dN[:, i].copy()
                B[0, 2 * i] = dx[0]
                B[1, 2 * i + 1] = dx[1]
                B[2, 2 * i] = dx[1]
                B[2, 2 * i + 1] = dx[0]

            Ke += (B.T @ D @ B) * detJ * weights[gp_id] * thickness

        for i in range(4):
            for j in range(4):
                for di in range(2):
                    for dj in range(2):
                        row = 2 * elem[i] + di
                        col = 2 * elem[j] + dj
                        rows[count] = row
                        cols[count] = col
                        data[count] = Ke[2 * i + di, 2 * j + dj]
                        count += 1
    return rows[:count], cols[:count], data[:count]

# ========================
# 应力计算 (2x2 高斯积分)
# ========================
@jit(nopython=True, parallel=True)
def calculate_stress(U, nodes, elements, E_values):
    stress_matrix = np.zeros((ny, nx, 3))
    gauss_points = np.array([[-0.57735, -0.57735], [0.57735, -0.57735],
                             [0.57735, 0.57735], [-0.57735, 0.57735]])
    weights = np.ones(4)

    for elem_id in prange(len(elements)):
        elem = elements[elem_id]
        u = np.zeros(8)
        for i in range(4):
            u[2*i:2*i+2] = U[2 * elem[i]:2 * elem[i] + 2]

        node_coords = nodes[elem].copy()
        E_e = E_values[elem_id]
        D = E_e / (1 - nu**2) * np.array([[1, nu, 0], [nu, 1, 0], [0, 0, (1 - nu) / 2]])

        stress_gauss = np.zeros((4, 3))

        for gp_id in range(4):
            xi, eta = gauss_points[gp_id]
            dN = 0.25 * np.array([
                [eta - 1, 1 - eta, 1 + eta, -eta - 1],
                [xi - 1, -xi - 1, xi + 1, 1 - xi]
            ])
            J = dN @ node_coords
            invJ = np.linalg.inv(J)
            B = np.zeros((3, 8))

            for i in range(4):
                dx = invJ @ dN[:, i].copy()
                B[0, 2 * i] = dx[0]
                B[1, 2 * i + 1] = dx[1]
                B[2, 2 * i] = dx[1]
                B[2, 2 * i + 1] = dx[0]

            strain = B @ u
            stress_gauss[gp_id] = (D @ strain.copy()).copy()

        i, j = elem_id // nx, elem_id % nx
        for k in range(4):
            stress_matrix[i, j, 0] += stress_gauss[k,0]
            stress_matrix[i, j, 1] += stress_gauss[k,1]
            stress_matrix[i, j, 2] += stress_gauss[k,2]

    stress_matrix /= 4.0
    return stress_matrix

# ========================
# 主程序
# ========================
if __name__ == "__main__":
    # 生成网格
    t0 = time.time()
    nodes, elements = generate_mesh(nx, ny, width, height)
    print(f"Mesh generated: {time.time() - t0:.2f}s")

    # 裂纹区域定义 (示例)
    crack_x_range = (20.0, 30.0)
    crack_y_range = (40.0, 60.0)
    crack_E_ratio = 1e-3  # 增大 E_ratio

    # 分配材料属性
    E_values = assign_material_properties(nodes, elements, crack_x_range, crack_y_range, crack_E_ratio)
    print(f"Material assigned: {time.time() - t0:.2f}s")

    # 组装刚度矩阵
    num_dofs = 2 * len(nodes)
    rows, cols, data = assemble_stiffness(nodes, elements, E_values, num_dofs)
    K = csr_matrix((data, (rows, cols)), shape=(num_dofs, num_dofs))
    print(f"Stiffness matrix assembled: {time.time() - t0:.2f}s")

    # 边界条件（固定底边中心点）
    center_bottom_node = np.where((nodes[:, 1] == 0.0) & (nodes[:, 0] == width / 2))[0]
    fixed_dofs = np.concatenate([2 * center_bottom_node, 2 * center_bottom_node + 1])

    # 载荷施加（顶部均布力）
    F = np.zeros(num_dofs)
    top_edges = elements[np.where(nodes[elements[:, 3], 1] == height)[0]]
    for elem in top_edges:
        n2, n3 = elem[2], elem[3]
        length = np.linalg.norm(nodes[n3] - nodes[n2])
        F[2 * n3 + 1] += applied_stress * length * thickness / 2
        F[2 * n2 + 1] += applied_stress * length * thickness / 2

    # 处理边界条件
    K = K.tolil()
    for fd in fixed_dofs:
        K[fd, :] = 0
        K[:, fd] = 0
        K[fd, fd] = 1.0
    F[fixed_dofs] = 0.0
    K = K.tocsr()

    print(f"Condition number of K: {np.linalg.cond(K.toarray())}")

    # 求解位移
    print("Solving system...")
    U = spsolve(K, F)
    print(f"Solution completed: {time.time() - t0:.2f}s")

    # 计算应力
    stress_matrix = calculate_stress(U, nodes, elements, E_values)
    print(f"Stress calculated: {time.time() - t0:.2f}s")

    # 计算主应力
    principal_stress = np.zeros((ny, nx))
    for i in prange(ny):
        for j in prange(nx):
            s = stress_matrix[i, j]
            s1 = (s[0] + s[1]) / 2 + np.sqrt(((s[0] - s[1]) / 2)**2 + s[2]**2)
            principal_stress[i, j] = s1

    # 输出结果
    print("\nStress Matrix Sample (5x5):")
    print(principal_stress[:5, :5])
    print(f"\nMax principal stress: {np.max(principal_stress):.2f} MPa")

    # 绘制主应力云图
    plt.imshow(principal_stress, cmap='jet', origin='lower', extent=[0, width, 0, height])
    plt.colorbar(label='Principal Stress (MPa)')
    plt.title('Principal Stress Distribution')
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.show()

Mesh generated: 0.32s
Material assigned: 1.22s
Stiffness matrix assembled: 8.96s
